https://www.statsmodels.org/dev/generated/statsmodels.tsa.holtwinters.ExponentialSmoothing.html

https://medium.com/analytics-vidhya/python-code-on-holt-winters-forecasting-3843808a9873



In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import pandas as pd
import numpy as np
import datetime as dt
import statsmodels.api as sm
import statsmodels.graphics as sg
import statsmodels.formula.api as smf
import statsmodels.stats as st


import matplotlib.pylab as plt

import seaborn as sns
sns.set_theme()

from sklearn.linear_model import LinearRegression
from sklearn import metrics 



%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
df = pd.read_csv('../data/nts.csv', index_col=0, parse_dates=True)
df.shape, df.columns

((42, 15),
 Index(['rynek', 'inflacja_r', 'inflacja_q', 'stopa_procentowa',
        'liczba_kredytow', 'tempo_wzrostu', 'ufnosc', 'duze_zakupy',
        'bezrobocie', 'spr_detaliczna', 'pkb', 'wynag', 'inflacja_q_kum',
        'ogolem', 'rpk'],
       dtype='object'))

In [66]:
df["quarter"] = pd.PeriodIndex(df.index, freq='Q').map(lambda r: int(str(r)[-1]))

drugiej połowie 2023 wynika z uruchomienia rządowego programu kredytów z preferencyjnym oprocentowaniem Bezpieczny Kredyt 2% (program był aktywny w okresie 01-07-2023 do 31-12-2023)

In [7]:
df.head()

,rynek,inflacja_r,inflacja_q,stopa_procentowa,liczba_kredytow,tempo_wzrostu,ufnosc,duze_zakupy,bezrobocie,spr_detaliczna,pkb,wynag,inflacja_q_kum,ogolem,rpk
2014-03-31,6193.211456,100.6,100.2,0.0446,41942.0,104.2,-20.5,-16.3,13.5,105.5,103.8,3882.041728,1.000000,11854.000000,0.0
2014-06-30,6227.984614,100.3,100.0,0.0447,45499.0,104.1,-16.8,-15.0,12.0,105.1,103.9,3922.805136,1.002000,10285.333333,0.0
2014-09-30,6357.260998,99.7,99.5,0.0443,43653.0,103.3,-16.9,-12.7,11.5,102.6,104.2,3948.311807,1.002000,11241.333333,0.0
2014-12-31,6438.874574,99.3,99.6,0.0388,42993.0,103.3,-13.8,-10.7,11.4,103.7,103.9,3986.074690,0.996990,14341.333333,0.0
2015-03-31,6473.616559,98.5,99.5,0.0348,42169.0,104.0,-14.9,-11.5,11.5,104.4,104.3,4028.086865,0.993002,10567.666667,0.0


In [8]:
bn, fn = 35, 42
train, test = df.iloc[:bn], df.iloc[bn:fn]
train.index, test.index, len(train), len(test)

(DatetimeIndex(['2014-03-31', '2014-06-30', '2014-09-30', '2014-12-31',
                '2015-03-31', '2015-06-30', '2015-09-30', '2015-12-31',
                '2016-03-31', '2016-06-30', '2016-09-30', '2016-12-31',
                '2017-03-31', '2017-06-30', '2017-09-30', '2017-12-31',
                '2018-03-31', '2018-06-30', '2018-09-30', '2018-12-31',
                '2019-03-31', '2019-06-30', '2019-09-30', '2019-12-31',
                '2020-03-31', '2020-06-30', '2020-09-30', '2020-12-31',
                '2021-03-31', '2021-06-30', '2021-09-30', '2021-12-31',
                '2022-03-31', '2022-06-30', '2022-09-30'],
               dtype='datetime64[ns]', freq=None),
 DatetimeIndex(['2022-12-31', '2023-03-31', '2023-06-30', '2023-09-30',
                '2023-12-31', '2024-03-31', '2024-06-30'],
               dtype='datetime64[ns]', freq=None),
 35,
 7)

In [90]:
X_train = train.drop('rynek', axis=1)
y_train = train['rynek']

X_test = test.drop('rynek', axis=1)
y_test = test['rynek']